In [ ]:
import os

from plotlyflask_server.data_parser import sort_RAFT_table as sRt
import pandas as pd
import numpy as np
import plotly.express as px
import re
from scipy.optimize import curve_fit
import itertools
from tqdm import tqdm

In [ ]:
# rectify/balance the table
sRt.df["t6h-conversion"] = np.nan
sRt.df["t10h-conversion"] = np.nan

# get all conversion headers, sort them by the hours
conversion_list = []
for column in sRt.df.columns:
    if "conversion" in column:
        conversion_list.append(column)

conversion_list.sort(key=lambda x: int(re.findall(r"\d+", x)[0]))

# get all the Mn and Mw headers
Mn_list = []
Mw_list = []
for column in sRt.df.columns:
    if "Mn" in column:
        Mn_list.append(column)
    if "Mw" in column:
        Mw_list.append(column)

print(conversion_list, Mn_list, Mw_list, sep="\n")

In [ ]:
def change_timeformat(time_format):
    h_m_s = str(time_format).split(":")
    m_format = int(h_m_s[0]) * 60 + int(h_m_s[1]) + int(h_m_s[2]) / 60
    return m_format

def change_timeformat_h(time_format):
    h_m_s = str(time_format).split(":")
    h_format = int(h_m_s[0]) + (int(h_m_s[1]) + int(h_m_s[2]) / 60) /60
    return h_format

# creating a table as a lookup to correct all sample measurement times

# get the hours from the headers with regex
hours_list = []
two_digit_regex = r"\d+"
for column in conversion_list:
    hours_list.append(int(re.findall(two_digit_regex, column)[0]))
hours_list.sort()

exact_times = pd.read_excel(sRt.INPUT_FILE_PATH, sheet_name="exact sampling times")

time_correction_df = pd.DataFrame(data=exact_times.iloc[3:12, 3:])
time_correction_df.columns = exact_times.loc[2][3:]

time_correction_df.reset_index(inplace=True, drop=True)
ext_time_corr_df_data =[]
for row in time_correction_df.iterrows():
    reactor_nr = row[1]["Reactor"]
    if type(reactor_nr) == int or reactor_nr == "closing of reactors": # that applies to reactor 15 and the closing of reactors
        row[1]["Reactor"] = str(reactor_nr)
        ext_time_corr_df_data.append(row[1])
    else: # that applies to all reactors which are described per row in pairs like "3+4"
        reactor_nr_s = (row[1]["Reactor"]).split("+")
        for reactor_nr in reactor_nr_s:
            row[1]["Reactor"] = reactor_nr
            ext_time_corr_df_data.append(row[1].copy())
ext_time_corr_df = pd.DataFrame(data=ext_time_corr_df_data, columns=exact_times.loc[2][3:])
ext_time_corr_df.reset_index(drop=True, inplace=True)
print("The timer was reset to 0 after the first closing of reactors in the \"t = 0\"-sampling-step")
ext_time_corr_df.columns = ["Reactor", 0, 1, 2, 4, 6, 8, 10, 15]

# change time format to minutes and set
time_cols = ext_time_corr_df.columns.difference(["Reactor"])
ext_time_corr_df[time_cols] = ext_time_corr_df[time_cols].apply(lambda x: [change_timeformat_h(d) for d in x])
ext_time_corr_df[0] = ext_time_corr_df[0].apply(lambda x: 0 )

# set index to reactor
ext_time_corr_df.set_index("Reactor", inplace=True)

ext_time_corr_df

In [ ]:
# replace the "sample determiner" with columns describing for the experiment number (next cell) and the actual reagents that made out the "determiner"
reaction_descriptors_dict = {}
abbreviation_keys = pd.read_excel(sRt.INPUT_FILE_PATH, sheet_name="Legend for Abbreviations")
abbreviation_keys.dropna(inplace=True)
for row in abbreviation_keys.itertuples():
    reaction_descriptors_dict[str(row.Symbol)] = row.Name

reaction_descriptors_dict

In [ ]:
# generate a reformatted table with all entries attributing to a sample analysis taken, described with a column of the right time
kinetic_curves = []
for index, polymerisation_kinetic in sRt.df.iterrows():

    kinetic_curve_entries = pd.DataFrame(index=range(len(polymerisation_kinetic[conversion_list])),
        data={"time" : hours_list, "conversion" : polymerisation_kinetic[conversion_list].values,
         "Mn" : polymerisation_kinetic[Mn_list].values, "Mw" : polymerisation_kinetic[Mw_list].values,
         # , "reactor" : polymerisation_kinetic["reactor"] reactor is not needed since the time is corrected
         })

    kinetic_curve_entries["exp_nr"] = str(polymerisation_kinetic["Experiment number"])
    kinetic_curve_entries["monomer"] = reaction_descriptors_dict[polymerisation_kinetic["monomer"]]
    kinetic_curve_entries["RAFT-Agent"] = reaction_descriptors_dict[polymerisation_kinetic["RAFT-Agent"]]
    kinetic_curve_entries["solvent"] = reaction_descriptors_dict[polymerisation_kinetic["solvent"]]

    # the times are dependent on the current reactor, get current
    current_reactor_nr = str(polymerisation_kinetic["reactor"])
    current_time_list = ext_time_corr_df.loc[current_reactor_nr]
    kinetic_curve_entries["time"] = list(current_time_list)

    kinetic_curves.append(kinetic_curve_entries)


In [ ]:
''' ToDo: no good scientific practice and also the experiment numbers where updated - this one should now have the number 505
 # manually exclude experiment 301 as it disturbs the covariance plotting and has an unreasonable conversion value course.
if kinetic_curves[301]["exp_nr"].iloc[1] == "301":
    print(kinetic_curves[301])
    fig = px.line(kinetic_curves[301].dropna(subset="conversion"), x="time", y="conversion", title=f"kinetic 301 ", labels={"x":"Time", "y":"Conversion"})
    fig.add_scatter(x=kinetic_curves[301]["time"], y=kinetic_curves[301]["conversion"], mode="lines+markers", line=dict(dash="dot"), name="Mn")
    fig.show()
    kinetic_curves.pop(301)
else:
    print("Cell was already run. Experiment is deleted.")
'''

In [ ]:
'''
deprecated: more theoretical accurate version but practically worse applicable
    # L is responsible for scaling the output range from [0,1] to [0,L]
    # b
    # k is responsible for scaling the input, which remains in (-inf,inf)
    # x0
def sigmoid (x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0)))+b
    return (y)

def sigmoid_derivative(x, L ,x0, k, b):
    y = (L*k*np.exp(-k*(x-x0)))/((np.exp(-k*(x-x0))+1)**2)
    return (y)


'''
def neg_growth(x, l, k):
    y = l * (1 - np.exp(k * (-x)))
    return y

def neg_growth_derivative(x, l, k):
    y = l * k * np.exp(k*(-x))
    return y

# as the Mn values do not all start at 0
def neg_growth_abscissae(x, l, k, b):
    y = l * (1 - np.exp(k * (-x))) + b
    return y

def linear_growth(x, m):
    y = m * x
    return y

def linear_growth_derivative(m):
    return m

'''
xrange = np.arange(-0.2, 5, 0.1)
example_fig = px.line()
for testparams in ([1,1], [2,1], [1,2]):
    example_fig.add_scatter(x=xrange, y=neg_growth(xrange, *testparams))
    example_fig.add_scatter(x=xrange, y=neg_growth_derivative(xrange, *testparams), opacity=0.5, line=dict(dash="dot"))

print("visualisation of the functions")
example_fig.show()
# '''
print("functions loaded")

In [ ]:
# checking out max and min of Mn/Mw
min_Mn, max_Mn = [], []
min_Mw, max_Mw = [], []

for kc in kinetic_curves:
    min_Mn.append(kc["Mn"].min()); max_Mn.append(kc["Mn"].max()); min_Mw.append(kc["Mw"].min()); max_Mw.append(kc["Mw"].max())

for _ in (min_Mn, max_Mn, min_Mw, max_Mw):
    _.sort()
    _ = [j/100000 for j in _]

molar_wg_comparison = px.line()
molar_wg_comparison.add_scatter(y=min_Mw, name="min Mw", line_color="darkgoldenrod", opacity=0.8, fill="tozeroy", fillcolor="orange")
molar_wg_comparison.add_scatter(y=max_Mw, name="max Mw", line_color="darkgoldenrod", opacity=0.8)
molar_wg_comparison.add_scatter(y=min_Mn, name="min Mn", line_color="firebrick", opacity=0.8, fill="tozeroy", fillcolor="red")
molar_wg_comparison.add_scatter(y=max_Mn, name="max Mn", line_color="firebrick", opacity=0.8)
molar_wg_comparison.update_layout(title="Mn and Mw comparison", yaxis_title=r"$\\text{Mn and Mw }[g/mol] \cdot 10^{-5}$")
molar_wg_comparison.show()

In [ ]:
fig = px.line(title="Kinetic Curve Fit")
colors = px.colors.qualitative.Plotly # set up a simple color palette
extended_xdata = np.linspace(-1, 16.5, 100) # x data array for plotting the fits


def add_fits_to_plot(figure, fit_func, fit_func_params,  fit_func_derivative=None, *args, **kwargs):
    figure.add_scatter(
        x=extended_xdata, y=fit_func(extended_xdata, *fit_func_params),
        opacity=1, line=dict(dash="dot"), name=f"{fit_func.__name__} fit", *args, **kwargs)
    if fit_func_derivative:
        figure.add_scatter(
            x=extended_xdata, y=fit_func_derivative(extended_xdata, *fit_func_params),
            opacity=0.3, line=dict(dash="dash"), name=f"{fit_func_derivative.__name__}", *args, **kwargs)

def fit_and_exclude_outliers(x, y, fit_func, p0, bounds, nan_policy="omit", iteration=1, outliers=None):
    outliers = outliers if outliers is not None else []

    # exclude the nan values from the data
    mask = ~np.isnan(y)
    x, y = x[mask], y[mask]

    cf_data = curve_fit(f=fit_func, xdata=x, ydata=y, p0=p0, nan_policy=nan_policy, maxfev=800 * 10, bounds=bounds)

    # calculate the fit points
    fit_points = np.array([fit_func(x, *cf_data[0]) for x in x])
    # calculate the standard deviation of the residuals between the fit and the data points
    sigma = np.std(fit_points - y)

    # exclude the outliers
    msk = ~(np.abs(fit_points - y) > 2 * sigma)
    if not msk.all():
        return fit_and_exclude_outliers(x=x[msk], y=y[msk], fit_func=fit_func, p0=cf_data[0], bounds=bounds, nan_policy=nan_policy, iteration=iteration+1, outliers=outliers + [x[~msk]])

    # calculate the squared error of fit and data points
    sq_err = np.mean(np.square(y - fit_points))

    result = {"x": x, "y": y, "p_opt" : cf_data[0], "p_cov" : cf_data[1], "sq_err": sq_err, "excluded_points" : (x[~msk],y[~msk]), "iteration" : iteration, "outliers" : outliers}
    return result

In [ ]:
kinetics_df = pd.DataFrame(columns=['exp_nr', 'max_con', 'theo_max_con', 'theo_react_end', 'monomer',
       'RAFT-Agent', 'solvent', 'fit_p1', 'fit_p2', 'p1_variance',
       'p1_p2_covariance', 'p2_variance', 'squared_error', 'conv_time_data',
       'Mn_time_data', 'Mw_time_data']) # create new dataframe with kinetics per row

for idx, kinetic_curve in enumerate(kinetic_curves):
    # first make sure the datapoints are in the right format and not sometimes int sometimes float
    xdata = np.array(kinetic_curve["time"].values, dtype=float)
    ydata_conv = np.array(kinetic_curve["conversion"].values, dtype=float)
    ydata_Mn = np.array(kinetic_curve["Mn"].values, dtype=float)/100000 # make it more comparable to conversion
    ydata_Mw = np.array(kinetic_curve["Mw"].values, dtype=float)/100000


    # fitting section for conversion
    p_initial = [max(ydata_conv), 0.1] # this is a mandatory initial guess
    ng_fit = fit_and_exclude_outliers(x=xdata, y=ydata_conv, fit_func=neg_growth, p0=p_initial, bounds=([0, -np.inf], [1, np.inf]))
    # l_fit = fit_and_exclude_outliers(x=xdata, y=ydata, fit_func=linear_growth, p0=[max(ydata)/7], bounds=([0], [np.inf]))

    popt, pcov = ng_fit["p_opt"], ng_fit["p_cov"]
    conv_time_data = np.array([ng_fit["x"], ng_fit["y"]])
    squared_error = ng_fit["sq_err"]

    # fitting section for Mn
    p_initial = [max(ydata_Mn[~np.isnan(ydata_Mn)]), 0.1, 0]
    ng_fit_Mn = fit_and_exclude_outliers(x=xdata, y=ydata_Mn, fit_func=neg_growth_abscissae, p0=p_initial, bounds=([0, -np.inf, -np.inf], [1, np.inf, np.inf]))

    popt_Mn, pcov_Mn = ng_fit_Mn["p_opt"], ng_fit_Mn["p_cov"]
    Mn_time_data = np.array([ng_fit_Mn["x"], ng_fit_Mn["y"]])
    squared_error_Mn = ng_fit_Mn["sq_err"]

    # fitting section for Mw
    ng_fit_Mw = fit_and_exclude_outliers(x=xdata, y=ydata_Mw, fit_func=neg_growth_abscissae, p0=p_initial, bounds=([0, -np.inf, -np.inf], [1, np.inf, np.inf]))

    popt_Mw, pcov_Mw = ng_fit_Mw["p_opt"], ng_fit_Mw["p_cov"]
    Mw_time_data = np.array([ng_fit_Mw["x"],ng_fit_Mw["y"]])
    squared_error_Mw = ng_fit_Mw["sq_err"]


    kinetics_df.loc[idx] = {"exp_nr":kinetic_curve["exp_nr"].iloc[1], "max_con":max(ydata_conv),
                        "theo_max_con":"yet to calc", "theo_react_end":"yet to calc",
                            "monomer":kinetic_curve["monomer"].iloc[1], "RAFT-Agent":kinetic_curve["RAFT-Agent"].iloc[1],
                            "solvent":kinetic_curve["solvent"].iloc[1],
                            "fit_p1":popt[0],"fit_p2":popt[1],
                            "p1_variance":pcov[0][0],"p1_p2_covariance":pcov[0][1],"p2_variance":pcov[1][1],
                            "squared_error":squared_error, "conv_time_data":conv_time_data,
                            "Mn_time_data":Mn_time_data, "Mw_time_data":Mw_time_data}

''' comment out here if plot is needed.
    marker_dict = dict(color=colors[idx%len(colors)]) # set same colors per kinetic
    fig.add_scatter(x=xdata, y=ydata, mode="lines+markers", opacity=1, name=kinetic_curve["exp_nr"].iloc[1], marker=marker_dict)
    add_fits_to_plot(neg_growth, neg_growth_derivative, popt, marker_dict)

fig.update_layout(
    yaxis=dict(
        range=[-0.1,1]
    ),

)
fig.show()
# '''

kinetics_df.reset_index(drop=True, inplace=True)
kinetics_df.drop(axis="index", index=kinetics_df[kinetics_df["max_con"] <= 0].index, inplace=True)
kinetics_df.reset_index(drop=True, inplace=True)
# kinetics_df

In [ ]:
# plot Mn Conversion difference
fig = px.line(title="Mn Conversion difference")
data_type_list = ["conv_time_data", "Mn_time_data", "Mw_time_data"]
data_list_name = ["conv over time", "Mn over time", "Mw over time"]
for idx, data in (enumerate(data_type_list)):
    for kin in kinetics_df[data].head(30):
        fig.add_scatter(x=kin[0], y=kin[1], mode="lines+markers", opacity=0.5, marker=dict(color=colors[idx%len(colors)]), showlegend=False, legendgroup=str(idx))
    # custom legend
    fig.add_scatter(x=[None], y=[None], marker=dict(color=colors[idx%len(colors)]), name= data_list_name[idx], legendgroup=str(idx))
# adding a threshold line because the SEC peaks smear into syspeak at that point
fig.add_scatter(x=[0, 16], y=[1000*10**(-5), 1000*10**(-5)], mode="lines", line=dict(dash="dash"), name="SEC limit", legendgroup="threshold")
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.05, xanchor="left", x=-0.1 ), yaxis_title=r"$\\text{Mn, Mw }[g/mol] \cdot 10^{-5}/ \ [ \%]$", xaxis_title="Time [h]")

In [ ]:
# normalize the errors by dividing them by their respective standard deviation
def normalize_errors(err):
    return err / np.std(err)

for error in ["squared_error", "p1_variance", "p2_variance", "p1_p2_covariance"]:
    kinetics_df[error] = normalize_errors(kinetics_df[error])

In [ ]:
# kinetics_df[kinetics_df["squared_error"]>0.01] 29/317 entries
# calculating covariance between errors to use only the reasonable ones

# permutate all combinations of the errors
errorcombs = [[],[],[]]
for err1, err2 in itertools.combinations(["squared_error", "p1_variance", "p2_variance", "p1_p2_covariance"], 2):
    # print(f"The covariance between {err1} and {err2} is {np.cov(kinetics_df[err1], kinetics_df[err2])}")
    # print(f"The correlation between {err1} and {err2} is {np.correlate(kinetics_df[err1], kinetics_df[err2])}")
    # print("\n")
    errorcombs[0].append(f"{err1}/{err2}")
    errorcombs[1].append(np.cov(kinetics_df[err1], kinetics_df[err2]))
    errorcombs[2].append(*np.correlate(kinetics_df[err1], kinetics_df[err2]))
errorcombs_dc = {"name": errorcombs[0], "covariance":errorcombs[1], "correlation":errorcombs[2]}
errorcombs_df = pd.DataFrame(data=errorcombs_dc)
errorcombs_df["i_correlation"] = errorcombs_df["correlation"] * (-1)

# plot bar plot per a pair of errors with superimposed correlation
err_fig = px.bar(title="Correlation between Errors", labels={"correlation":"correlation"}, log_y=True)
err_fig.add_bar(x=errorcombs_df["name"], y=errorcombs_df["correlation"], name="positive correlation", marker_color="green")
err_fig.add_bar(x=errorcombs_df["name"], y=errorcombs_df["i_correlation"], name="negative correlation", marker_color="crimson")
err_fig.show()

In [ ]:
# violin plots of the errors
for errors in ["p1_variance", "p2_variance", "p1_p2_covariance"]:
    kinetics_df[errors] = kinetics_df[errors].apply(lambda x: np.absolute(x))
fig = px.violin(kinetics_df, y=["squared_error", "p1_variance", "p2_variance", "p1_p2_covariance"], box=True, points="all", log_y=True)
fig.update_layout(title="Errors normalize by \u03C3", xaxis_title="Error type", yaxis_title="value", yaxis=dict(title="log(value)", range=(-22, 3)))
fig.show()

In [ ]:
# get information about within which error margin the fits and therefore kinetics are good
# split error in quartiles, index per error

title_dic = {"squared_error":"r² - error",
             "p1_variance":"variance of parameter 1",
             "p2_variance":"variance of parameter 2",
             "p1_p2_covariance":"covariance (Params 1 & 2)"}

def get_quartile_indexes(error_type):
    quartile_len = len(kinetics_df) / 4
    quartile_ranges = np.array([(a * quartile_len, (a + 1) * quartile_len) for a in range(4)], dtype=int)
    quartiles_list = list()
    for q in range(4):
        quartiles_list.append(kinetics_df.sort_values(by=[error_type]).iloc[quartile_ranges[q][0]:quartile_ranges[q][1]])
    return quartiles_list

def export_quartile_figures():
    for err in tqdm(["squared_error", "p1_variance", "p2_variance", "p1_p2_covariance"]):
        err_quartiles = get_quartile_indexes(err)

        # plot the single quartiles
        for nr, n in enumerate(err_quartiles):
            iks = n.index
            fg = px.line(title=f"kinetics (quartile {nr+1} of the {title_dic[err]}) ", labels={"x":"time", "y":"conversion"})
            for ik in iks:
                marker_dict = dict(color=colors[ik%len(colors)])
                fit_data = kinetics_df.iloc[ik]
                xdt, ydt = fit_data["xdata"], fit_data["ydata"]
                fg.add_scatter(x=xdt, y=ydt, mode="lines+markers", name=kinetic_curves[ik]["exp_nr"].iloc[0], marker=marker_dict)
                add_fits_to_plot(fig, neg_growth, neg_growth_derivative, [fit_data["fit_p1"], fit_data["fit_p2"]], marker=marker_dict)
                fg.update_layout(yaxis=dict(range=[-0.1,1]), xaxis_title="Time [h]", yaxis_title="Conversion [%]")
            fg.write_image(f"data exploration figures/{err} ({nr+1} quartile).svg")

# export_quartile_figures() ; print("exported")

In [ ]:
# descry when a function aligns to the datapoints in a reasonable way
#   Hence, wheneth' the blunder exaggerates, an 80% betweeneth' of the maximum conversion in that kinetic should be assessed to be the maximum conversion.
#   let's give a point for every quartile further from the first for the single errors divided by the maximum score (that is 3*4=12)
error_list = ["squared_error", "p1_variance", "p2_variance", "p1_p2_covariance"]
error_dic = {}
score = np.zeros(len(kinetics_df), int)
for error in error_list:
    quartiles = get_quartile_indexes(error)
    # for every error we want to give an error per index
    for sc, quartile in enumerate(quartiles):
        if sc == 0:
            continue
        # each quartile is a dataframe, where the latter one raise a higher error ( 0, 1, 2, 3)
        for num in quartile.index:
            score[num]+=sc
kinetics_df["error_score"]=score

In [ ]:
p1_values = kinetics_df["fit_p1"].values
average_conversion = p1_values.mean()
min_con, max_con = p1_values.min(), p1_values.max()

print(f"According to the fits the mean maximum conversion is {average_conversion:.3n}, the minimum maximum is {min_con:.3n} and the maximum overall is {max_con:.3n}")

In [ ]:
# determining apposite reaction end values.
# The moment where 90% of the maximum conversion have been reached can be seen as a practical maximum conversion point
kinetics_df["theo_max_con"] = kinetics_df["fit_p1"].apply(lambda x: x*0.9)

def y_converted_negative_growth(y, l, k):
    return -np.log(1 - y / l) / k

kinetics_df["theo_react_end"] = \
    [y_converted_negative_growth(y, fit_p1, fit_p2) for y, fit_p1, fit_p2 in zip(kinetics_df["theo_max_con"], kinetics_df["fit_p1"], kinetics_df["fit_p2"])]

# the theoretical maximal conversion must be capped at reasonable time (we take two days here) that is applying 139/313 entries
kinetics_df["theo_react_end"] = [30 if x > 30 else x for x in kinetics_df["theo_react_end"].values]
# recalculate the apposite maximal conversion
kinetics_df["theo_max_con"] = [neg_growth(x, p1, p2) for x, p1, p2 in zip(kinetics_df["theo_react_end"], kinetics_df["fit_p1"], kinetics_df["fit_p2"])]


In [ ]:
# searching manually for interesting experiments to be reproduced by MR.
search_queue_monomer = kinetics_df["monomer"].apply(lambda x: x in ["Styrene",  "4-Methylstyrene", "Benzyl acrylate"])
search_queue_solvent = kinetics_df["solvent"].apply(lambda x: x in ["Dimethylformamide"])
kinetics_df[search_queue_monomer & search_queue_solvent]

In [ ]:
# one longrun test for the theo react end cap (146) for up to 72h
# two to see reproducible exp from within the second quantile in average (score of 4/5)
#   one "fast" fit point (202)
#   one for a weak conversion (90)
kinetics_df[kinetics_df['exp_nr'].isin(["205", "90", "146"])]

In [ ]:
# to find the optimal threshold parameters for search one has to keep in mind that with high conversion (assuming around 80%, research/citation needed) increasing side reaction take place. After that the reactions should be sorted after time then error score. Maybe a multiple-decreasing-threshold-sorting-algorithm would be good.
# so first priority would be sorting after nearest to 80% conversion.

# create a score ingesting the importance of the different kinetic descriptors
    # Conversion*1 + time*(-0.8) + error_score*(0.5)
    # while spanning between the optimum and the least bearable values like in the following:
        #  Conversion: |con-0.8| - 0 (0.8 is the optimum)
        #     using a linear decreasing function -x*m+b
        #  Time: 0 - np.inf (0 is the optimum) (more than 72 is not bearable)
        #     using a negative potential function -x**2+b
        #  Error: 0 - 12 (0 is the optimum) (the error is more negligible)
        #     using a linear decreasing function -x*m+b
score = []
for row in kinetics_df.itertuples():
    score.append(((0.8-np.abs(row.theo_max_con - 0.8))/0.8*1 + (-(row.theo_react_end/72)**2+1)*0.8 + ((12-row.error_score)/12)*0.5))
kinetics_df["score"] = score
kinetics_df

# Interface:
- Give in Monomer ~~and Mn~~
    - The dataset will then be filtered for that one and the results, sorted via conversion (100% conversion, 80% time, 50% Fit) be given out.
        - Tabular data with kinetic curve and fit as hover, reaction properties (solvent, Raft agent, time...)
- Table should be interactive,
- optionally, solvent and or raft agent can be chosen
- Mn search is also just optionally after "main" search, as the Mn data is hard to plot...
- if the queued monomer is not in the dataset the closest ones should be searched for and displayed instead.
- same goes with solvent via polarity comparison maybe just as a highly experimental suggestion: "Suggestion: Less polar solvents than Toluene might prove more useful but are not tried out in the dataset."

In [ ]:
def color_variant(hex_color, brightness_offset=1):
    """ takes a color like #87c95f and produces a lighter or darker variant """
    if len(hex_color) != 7:
        raise Exception("Passed %s into color_variant(), needs to be in #87c95f format." % hex_color)
    rgb_hex = [hex_color[x:x+2] for x in [1, 3, 5]]
    new_rgb_int = [int(hex_value, 16) + brightness_offset for hex_value in rgb_hex]
    new_rgb_int = [min([255, max([0, i])]) for i in new_rgb_int] # make sure new values are between 0 and 255
    # hex() produces "0x88", we want just "88"; also we zfill to pad with leading zeros if necessary, e.g. 9 -> 09
    return "#" + "".join([hex(i)[2:].zfill(2) for i in new_rgb_int])

In [ ]:
# Some search functions:
def search_for_exp(exp_nr: str | list) -> pd.DataFrame:
    if type(exp_nr) == str:
        print(f"Searching for experiment {exp_nr}")

        return kinetics_df[kinetics_df["exp_nr"] == exp_nr]
    else:
        return kinetics_df[kinetics_df["exp_nr"].isin(exp_nr)]

def plot_exp(exp_nr: str | list, plot_mn: bool=False, plot_mw: bool=False, fit_curves: tuple=(True, True)):
    plot_data = search_for_exp(exp_nr)
    exp_fig = px.line(title=f"Kinetic Curve Fit for {exp_nr}")
    for kinetic_to_plot in plot_data.itertuples():
        x_data, ydata = kinetic_to_plot.conv_time_data
        marker_dict = dict(color=colors[int(kinetic_to_plot.Index)%len(colors)])
        exp_fig.add_scatter(x=x_data, y=ydata, mode="lines+markers", name=kinetic_to_plot.exp_nr, marker=marker_dict, legendgroup=str(kinetic_to_plot.exp_nr))
        if fit_curves[0]:
            if fit_curves[1]:
                add_fits_to_plot(exp_fig, neg_growth, [kinetic_to_plot.fit_p1, kinetic_to_plot.fit_p2], fit_func_derivative=neg_growth_derivative, marker=marker_dict, legendgroup=str(kinetic_to_plot.exp_nr), showlegend=False)
            else:
                add_fits_to_plot(exp_fig, neg_growth, [kinetic_to_plot.fit_p1, kinetic_to_plot.fit_p2], marker=marker_dict, legendgroup=str(kinetic_to_plot.exp_nr), showlegend=False)

        if plot_mn:
            marker_dict["color"] = color_variant(marker_dict["color"], 30)
            x2_data, y2_data = kinetic_to_plot.Mn_time_data
            exp_fig.add_scatter(x=x2_data, y=y2_data, mode="lines+markers", name="Mn of " + kinetic_to_plot.exp_nr, marker=marker_dict, legendgroup=str(kinetic_to_plot.exp_nr))
        if plot_mw:
            marker_dict["color"] = color_variant(marker_dict["color"], -60)
            x2_data, y2_data = kinetic_to_plot.Mw_time_data
            exp_fig.add_scatter(x=x2_data, y=y2_data, mode="lines+markers", name="Mw of " + kinetic_to_plot.exp_nr, marker=marker_dict, legendgroup=str(kinetic_to_plot.exp_nr), opacity=0.5)

    exp_fig.update_layout(yaxis=dict(range=[-0.1,1]), xaxis_title="Time [h]", yaxis_title="Conversion [%]")
    exp_fig.show()

def find_optimal_synthesis(monomer: str | list):
    search_q_monomer = [x in monomer for x in kinetics_df["monomer"]]
    result_df = kinetics_df[search_q_monomer].sort_values(by=["score"], ascending=False)
    return result_df

def refine_search(dataframe: pd.DataFrame, monomer: list = None, solvent: list = None, raft_agent: list = None):
    len_df = len(dataframe)
    search_q_monomer, search_q_solvent, search_q_raft_agent = [np.array([True] * len_df) for _ in range(3)]
    if monomer:
        search_q_monomer = dataframe["monomer"].apply(lambda x: x in [*monomer])
    if solvent:
        search_q_solvent = dataframe["solvent"].apply(lambda x: x in [*solvent])
    if raft_agent:
        search_q_raft_agent = dataframe["RAFT-Agent"].apply(lambda x: x in [*raft_agent])
    return dataframe[search_q_monomer & search_q_solvent & search_q_raft_agent]

# search_for_exp(["145", "90", "253"])
# plot_exp(["145", "90", "253"], True, True)
plot_exp(["241", "146", "392"], True, True)
# find_optimal_synthesis(["Styrene", "4-Methylstyrene", "Benzyl acrylate"])
# refine_search(kinetics_df, solvent=["Dimethylformamide"], monomer=["Styrene", "4-Methylstyrene", "Benzyl acrylate"], raft_agent=["2-Cyan-2-propylbenzodithioat"])
# refine_search(find_optimal_synthesis(["Styrene", "4-Methylstyrene", "Benzyl acrylate"]), solvent=["Dimethylformamide"], raft_agent=["2-Cyan-2-propylbenzodithioat"])

In [ ]:
import panel as pn


In [ ]:
pn.extension()
wind_speed = pn.widgets.FloatSlider(
    value=5, start=0, end=20, step=1, name="Wind Speed (m/s)"
)

efficiency = 0.3

power = wind_speed.rx() * efficiency

power_text = pn.rx(
    "Wind Speed: {wind_speed} m/s, "
    "Efficiency: {efficiency}, "
    "Power Generation: {power:.1f} kW"
).format(wind_speed=wind_speed, efficiency=efficiency, power=power)

pn.Column(power_text).servable()

In [ ]:
from IPython.display import Markdown, display
def display_color(hex_color, width=6):
    if type(hex_color) == str:
        display(Markdown(f'<span style="font-family: monospace">{hex_color} <span style="color: {hex_color}">{chr(9608)*width}</span></span>'))
    else:
        display(Markdown('<br>'.join(
            f'<span style="font-family: monospace">{color} <span style="color: {color}">{chr(9608)*width}</span></span>'
            for color in hex_color)))
display_color(colors)

In [ ]:
# manually typing in the proof experimental data from michael
# "MRG-046-G-ZL-1-E-DMF" -> exp
# "MRG-046-G-ZL-4-B-DMF" -> exp
# "MRG-046-G-ZL-15-F-DMF" -> exp
# "exp", "monomer", "RAFT-Agent", "solvent", "time", "conversion", "Mn", "Mw"


""" !The experiment numbers changed to from 146, 90 and 205 to 241, 145 and 343!
M. Ringleb:
146 evaluation of the observations was done after ca. 90 hno addition of NMR solvent and SEC eluent to vials for 15 h sample -->
potential eNAplanation for problems with evaluation or deviations (sample stood for 5 h without being quenched) --> for NMR additon of 400 yL of CDCl3 prior to filling to NMR tube

90 evaluation of this points was done after 20 h time in the reactor
double addition of NMR solvent and SEC eluent to the vials for t =10h
also for NMR sampling for t=10 h sample the septum was pushed through the lid, so the vial stood open for ca. 9hours before it was filled to the NMR tube "

205 evaluation of this points was done after 20 h time in the reactor"""

proof_data = [
    ["241b", "1", "E", "DMF", 0, 0.0, 0.0, 0.0],
    ["241b", "1", "E", "DMF", 15, 0.05, 3000, 3400],
    ["241b", "1", "E", "DMF", 60, 0.24, 3500, 3900],
    ["241b", "1", "E", "DMF", 72, 0.24, 3500, 4100],
    ["241b", "1", "E", "DMF", 84, 0.24, 3700, 4200],
    ["145b", "4", "B", "DMF", 0, 0.0, 0.0, 0.0],
    ["145b", "4", "B", "DMF", 2.95, 0.02, 870, 970],
    ["145b", "4", "B", "DMF", 10, 0.10, 2200, 2500],
    ["343b", "15", "F", "DMF", 0, 0.0, 0.0, 0.0],
    ["343b", "15", "F", "DMF", 5.62, 0.85, 7700, 14800],
    ["343b", "15", "F", "DMF", 8, 0.90, 7100, 15100],
    ]

proof_df_points = pd.DataFrame(data=proof_data, columns=["exp_nr", "monomer", "RAFT-Agent", "solvent", "time", "conversion", "Mn", "Mw"])
# reformatting to kinetics_df format
proof_df = pd.DataFrame()
for kinetic_nr in proof_df_points["exp_nr"].unique():
    kinetic = proof_df_points[proof_df_points["exp_nr"] == kinetic_nr]
    proof_kinetic = pd.DataFrame({"exp_nr":kinetic["exp_nr"].iloc[0],
                                  "monomer":reaction_descriptors_dict[kinetic["monomer"].iloc[0]],
                                  "RAFT-Agent":reaction_descriptors_dict[kinetic["RAFT-Agent"].iloc[0]],
                                  "solvent":reaction_descriptors_dict[kinetic["solvent"].iloc[0]],

                                  "conv_time_data":[[kinetic["time"].values, kinetic["conversion"].values]],
                                  "Mn_time_data":[[kinetic["time"].values, kinetic["Mn"].values/100000]],
                                  "Mw_time_data":[[kinetic["time"].values, kinetic["Mw"].values/100000]]})

    proof_df = pd.concat([proof_df, proof_kinetic])
proof_df

In [ ]:
# comparing proof and original experiments data
fig = px.line()

for idx, data in (enumerate(data_type_list)):
    comb_d = pd.concat([proof_df, search_for_exp(["241", "145", "343"])])
    for kin, exp_nr in zip(comb_d[data], comb_d["exp_nr"]):
        fig.add_scatter(x=kin[0], y=kin[1], mode="lines+markers", opacity=0.5, name=exp_nr, marker=dict(color=colors[idx%len(colors)]), showlegend=False, legendgroup=str(idx))
    # custom legend
    fig.add_scatter(x=[None], y=[None], marker=dict(color=colors[idx%len(colors)]), name=data_list_name[idx], legendgroup=str(idx))
# adding a threshold line because the SEC peaks smear into syspeak at that point
fig.add_scatter(x=[0, 84], y=[1000*10**(-5), 1000*10**(-5)], mode="lines", line=dict(dash="dash"), name="SEC limit", legendgroup="threshold")
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.05, xanchor="left", x=-0.1 ), yaxis_title=r"$Mn, Mw \ [g/mol] \cdot 10^{-5}\ and \ conversion \ [ \%]$", xaxis_title="Time [h]")
fig.show()
"""
Y. Köster:
The Comparison of proof and prior data shows that the experimental data is not easily reproducible with respective accuracy in small orders of magnitude (Conversion after 15 h around 10 %)
Especially for the RAFT synthesis with this concern of low reaction rate an extrapolation further than double the time (to 30 h) seems not feasible.
 Therefore the former time cap of 70 h gets changed to 30 h.
"""

In [ ]:
# search for all kinetics with falling Mn, Mw Values, they start appearing a lot when the Mn is over 0.25 * 10**5
search_q_falling_Mn = [True if np.mean(kin[1]) > 0.45 else False for kin in kinetics_df["Mn_time_data"]]
# plot_exp(kinetics_df[search_q_falling_Mn]["exp_nr"].values, True, True,  fit_curves=(False, False))
print("The sinking Mn/Mw got all curated out") # ToDo plot them via the discarded data

In [ ]:
# how many number unique did it actually make through curation
interests = ["monomer", "RAFT-Agent", "solvent"]
for interest in interests:
    print(str(kinetics_df[interest].nunique()) + " for " + interest)
    print(kinetics_df[interest].unique())

In [ ]:
refine_search(dataframe=kinetics_df, raft_agent=["Benzyl 1H-pyrrole-1-carbodithioate"])


In [ ]:
# plot_exp(["241", "145", "343"], True, True)
search_for_exp(["241", "145", "343"])

In [ ]:
# trying to parse PSMILES to explicit SMILES
from rdkit import Chem
PSMILES = "[*]CC([*])(C)(C(=O)OC)"
explicit_smiles = "[CH2][C](C)(C(=O)OC)"
# stripped_PSMILES = PSMILES.replace("[*]", "")
stripped_PSMILES = "CC(C)(C(=O)OC)"


PSMILES_mol = Chem.MolFromSmiles(PSMILES)
explicit_smiles_mol = Chem.MolFromSmiles(explicit_smiles)
stripped_PSMILES_mol = Chem.MolFromSmiles(stripped_PSMILES)

for atom in PSMILES_mol.GetAtoms():
    print(str(atom.GetIdx()) +" "+ atom.GetSymbol() + " has " + str(atom.GetExplicitValence()) + " explicit valence")
print(PSMILES, " PSMILES")
display(PSMILES_mol)

for atom in explicit_smiles_mol.GetAtoms():
    print(str(atom.GetIdx()) +" "+ atom.GetSymbol() + " has " + str(atom.GetExplicitValence()) + " explicit valence")
print(explicit_smiles, " explicit_smiles")
display(explicit_smiles_mol)

for atom in stripped_PSMILES_mol.GetAtoms():
    print(str(atom.GetIdx()) +" "+ atom.GetSymbol() + " has " + str(atom.GetExplicitValence()) + " explicit valence")
print(stripped_PSMILES, " stripped_PSMILES")
display(stripped_PSMILES_mol)

In [ ]:
''' illegal codes
* C=C* -> *C=C*
*OC(CCC(c1ccccc1))CC(=O)O*
*C=CC(=O)OC#N*
*CC(=C)C#N*
CC(c1cc(Cl)cccc1)
'''

Chem.MolFromSmiles("S"+"CC(c1cc(Cl)cccc1)"*2+"S")

In [ ]:
import re
def parse_p_to_explicit_smiles(psmiles: str) -> str:
    # first create a mol object
    # print(f"parsing {psmiles}")

    mol = Chem.MolFromSmiles(psmiles)
    if mol is None:
        print(f"{psmiles} is not a valid SMILES string")
        return ""


    # iterate over the atoms catching the position of the C Atoms following and preceding [*] and their valence
    star_index = []
    valence = []
    atomsymbols = []
    for atom in mol.GetAtoms():
        atomsymbol = atom.GetSymbol()
        if atom.GetIsAromatic():
            atomsymbol = atomsymbol.lower()
        atomsymbols.append(atomsymbol)
        if atomsymbol == "*":
            star_index.append(atom.GetIdx())
        valence.append(atom.GetExplicitValence())
    c_index = [star_index[0]+1, star_index[1]-1] # cannot do that atom symbols are not in order

    if len(star_index) > 2:
        # raise a type error
        raise TypeError ("{__name__} cannot handle ladder polymers/more than two [*] in the PSMILES string!")
    # the current valence of the C atoms is the explicit minus the [*] connection/-1
    def valence_change(val):
        return 4 - val
    h_num = []
    for sy, val in zip(atomsymbols, valence):  # number of H atoms assuming C atoms
        if sy == "C":
            h_num.append(valence_change(val))
        else:
            h_num.append(0)

    # replace C atoms with the explicit hydrogen count and catch the special case of parenthesised like e.g.([*]) and parenthesise the dangling part of the smiles string to the end after that C atom like [*]CC([*])(C)C(=O)OC -> [CH2][C](C)(C(=O)OC) (attention to the bracket pair ending with the last symbol ")")

    atomsymbols = [_atom if _atom != "*" else r"\*" for _atom in atomsymbols] # escape the * for the regex

    # create a dict of atom index and index of it's symbol in the string
    atom_positions = [m.start() for m in re.finditer(r"("+'|'.join(atomsymbols)+")", psmiles)]


    atom_string_map = {atom_symbol_nr:string_pos for atom_symbol_nr, string_pos in zip(range(len(atomsymbols)), atom_positions)}
    c_str_indexes = [atom_string_map[_c_index] for _c_index in c_index]

    c_str_h_num = {atom_string_map[_c_index]:h_num[_c_index] for _c_index in c_index}

    smiles_reconstruction = ""
    for idx, character in enumerate(psmiles):
        if idx in c_str_indexes:
            smiles_reconstruction += f"[CH{c_str_h_num[idx]}]" if c_str_h_num[idx] > 0 else f"[{character}]"
        else:
            smiles_reconstruction += character


    smiles_reconstruction = smiles_reconstruction.replace("([*])", "")
    smiles_reconstruction = smiles_reconstruction.replace("[*]", "")
    smiles_reconstruction = smiles_reconstruction.replace("*", "")

    return smiles_reconstruction
for test_PSMILE in ["*C(=O)CC(CCC)O*", "[*]CC([*])(C)(C(=O)OC)", "*OC(CCC(c1ccccc1))CC(=O)O*",]:
# for test_PSMILE in ["*OC(CCC(c1ccccc1))CC(=O)O*", "*C(=O)CC(CCC)O*", "[*]CC([*])(C)(C(=O)OC)", ]:
    print(test_PSMILE + " parsing...")
    print(parse_p_to_explicit_smiles(test_PSMILE))


In [ ]:
downloads_p = r"C:\Users\fo72qev\Downloads"
homo_p = downloads_p + r"\new_final_homo.csv"
copo_p = downloads_p + r"\copo_final.csv"

In [ ]:
for dt_p in [homo_p, copo_p]:
    dt = pd.read_csv(dt_p)
    psmiles_columns = [column for column in dt.columns if "psmiles" in column.lower()]
    for psmiles_column in psmiles_columns:
        dt[psmiles_column+"to_explicit_SMILES"] = dt[psmiles_column].apply(lambda y: parse_p_to_explicit_smiles(y))
    dt.to_csv(dt_p.replace(".csv", "_explicit_SMILES.csv"))

In [ ]:
print(pd.read_csv(homo_p))